In [14]:
import os
os.getcwd()

'/project/6005917/linxin67/Projects/MambaPose/Video_Pose/3_VideoMambaPose/src/models/experiments/mamba_encoder'

In [15]:
os.chdir("/home/linxin67/projects/def-btaati/linxin67/Projects/MambaPose/Video_Pose/3_VideoMambaPose/src/models/experiments/mamba_encoder")
os.getcwd()

'/project/6005917/linxin67/Projects/MambaPose/Video_Pose/3_VideoMambaPose/src/models/experiments/mamba_encoder'

In [16]:
!pip list

Package                   Version                   Editable project location
------------------------- ------------------------- ------------------------------------------------------------------------------------------
anyio                     4.3.0+computecanada
argon2_cffi               23.1.0+computecanada
argon2_cffi_bindings      21.2.0+computecanada
arrow                     1.3.0+computecanada
asttokens                 2.4.1+computecanada
async_lru                 2.0.4+computecanada
attrs                     23.2.0+computecanada
Babel                     2.15.0
beautifulsoup4            4.12.3+computecanada
bleach                    6.1.0+computecanada
blinker                   1.8.2
causal-conv1d             1.0.0                     /project/6005917/linxin67/Projects/MambaPose/Video_Pose/2_HeatMap/VideoMamba/causal-conv1d
certifi                   2023.7.22+computecanada
cffi                      1.15.1+computecanada
charset_normalizer        3.2.0+computecanada
click     

In [17]:
import os
import torch
import torch.nn as nn
from functools import partial
from torch import Tensor
from typing import Optional
import torch.utils.checkpoint as checkpoint
 
# remember that this is einstein operation, which is the special fancy way of reshaping.
from einops import rearrange
from timm.models.vision_transformer import _cfg
from timm.models.registry import register_model
from timm.models.layers import trunc_normal_

from timm.models.layers import DropPath, to_2tuple
from timm.models.vision_transformer import _load_weights

import math

from mamba_ssm.modules.mamba_simple import Mamba

In [19]:
# generating a random input
# (Batch, Channel number, NumFrames, W, H) = (16, 3, 8, 224, 224)

# Define the dimensions
batch_size = 16
num_frames = 8
height = 224
width = 224
channels = 3

# Generate a random tensor
test_video = torch.rand(batch_size, channels, num_frames, height, width) # I get an error .... 384, 3, 1, 16, 16

# Check the shape of the random tensor
print("Shape of the random tensor:", random_video.shape)


Shape of the random tensor: torch.Size([16, 8, 3, 224, 224])


#### running the video preprocessing

#### running the mamba model

In [21]:
import VideoMamba as vm

In [22]:
class VideoMambaPose(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = vm.videomamba_tiny() # TODO this is temporary 

    def forward(self, x):
        x = self.encoder(x)

        # adding my own layer, see how deciwatch did it.
        print(x.shape)


In [24]:
test_model = VideoMambaPose()

y = test_model(test_video)

Use checkpoint: False
Checkpoint number: 0


RuntimeError: No CUDA GPUs are available